# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [87]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [4]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
an3_fold = f'{drive}:/data/big2/BD3/an'
raw3_fold = f'{drive}:/data/big2/BD3/raw'

# Load
- 등록&제원 -> result 5등급

## 등록정보(STD_CEG_CAR_MIG) 5등급만

In [13]:
# 8.6s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0505' OR EXHST_GAS_GRD_CD = 'A05T5';")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [14]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378646 entries, 0 to 2378645
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   VIN             object
 1   BSPL_STDG_CD    int64 
 2   VHCL_ERSR_YN    object
 3   MANP_MNG_NO     object
 4   YRIDNW          int64 
 5   VHCTY_CD        object
 6   PURPS_CD2       object
 7   FRST_REG_YMD    int64 
 8   VHCL_FBCTN_YMD  object
 9   VHCL_MNG_NO     object
dtypes: int64(3), object(7)
memory usage: 181.5+ MB


In [15]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [16]:
carr = car.rename(columns=car_ch_col)

In [17]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [18]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [19]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [20]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [21]:
# 3.0s
att = wb.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1010946 non-null  object
 1   RDCDVC_SE_CD  1010946 non-null  object
dtypes: object(2)
memory usage: 15.4+ MB


In [22]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [23]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [39]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_CD, STDG_SGG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_CD  46333 non-null  int64 
 2   STDG_SGG_CD   46333 non-null  int64 
 3   STDG_CTPV_NM  46333 non-null  object
 4   STDG_SGG_NM   46310 non-null  object
dtypes: int64(3), object(2)
memory usage: 1.8+ MB


In [40]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_CD', 'STDG_SGG_CD', 'STDG_CTPV_NM',
       'STDG_SGG_NM'],
      dtype='object')

In [41]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드', 
    'STDG_SGG_CD':'시군구코드',
}

In [42]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도코드', '시군구코드', '시도', '시군구'], dtype='object')

## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [28]:
# 2.4s
aea = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  804082 non-null  object
 1   ELPDSRC_STTS_CD      804082 non-null  object
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object
dtypes: object(3)
memory usage: 18.4+ MB


In [29]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [30]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [31]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [32]:
aear.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMCXLS7BP2U626435,A32Y,Y
1,KNAJA5265VA794691,A32Y,Y
2,KPBLC3D81VP018295,A32Y,Y
3,KPDHADMD1YP225493,A32Y,Y
4,KNAMA75122S249083,A32Y,Y


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [33]:
# 1.8s
lgv = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373371 entries, 0 to 373370
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  373371 non-null  object
 1   ELPDSRC_STTS_CD      373371 non-null  object
 2   ELPDSRC_LST_APRV_YN  160690 non-null  object
dtypes: object(3)
memory usage: 8.5+ MB


In [34]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [35]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [36]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [37]:
lgvr.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMHSD81VP4U791871,A32M,NaN
1,KPBFA2AF16P223800,A32M,NaN
2,KMFWVH7HP7U772891,A32M,NaN
3,KMFXKN7BPYU361960,A32M,NaN
4,KMFXKS7BP2U563724,A32M,NaN


## 비상시 및 계절제 단속발령(N_IS_ISSUE_DISCLOSURE)

In [ ]:
# # 1.8s
# isdis = we.export_to_pandas("SELECT GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD FROM ODS_N_IS_ISSUE_DISCLOSURE;")
# isdis.info()

In [124]:
# about 2.1s
name = 'ODS_N_IS_ISSUE_DISCLOSURE_1'
file_name = f'{name}.csv'
isdis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['REGLT_NO', 'GNFD_NO', 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD'], nrows=100**2) # usecols=['REGLT_NO', 'GNFD_NO, 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD']
isdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   REGLT_NO       10000 non-null  int64 
 1   GNFD_NO        10000 non-null  int64 
 2   REG_SIDO_CD    10000 non-null  int64 
 3   REG_SIGNGU_CD  10000 non-null  int64 
 4   VIN            10000 non-null  object
dtypes: int64(4), object(1)
memory usage: 390.8+ KB


In [125]:
isdis.head()

,REGLT_NO,GNFD_NO,REG_SIDO_CD,REG_SIGNGU_CD,VIN
0,54793831,1838,41,190,KMFXKS7BP3U657943
1,54793839,1838,41,190,KNAMA75432S388470
2,54793843,1838,28,237,KNGDNM9N14K178854
3,54795717,1838,41,130,KMFGA17PP7C072098
4,54793837,1838,41,190,KPBSH3AB18P082348


In [126]:
isdis.columns

Index(['REGLT_NO', 'GNFD_NO', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'VIN'], dtype='object')

In [127]:
isdis_ch_col = {
    'REGLT_NO':'적발번호', 
    'GNFD_NO':'발령번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'VIN':'차대번호',
}

In [128]:
isdisr = isdis.rename(columns=isdis_ch_col)
isdisr.columns

Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '차대번호'], dtype='object')

## 운행제한 발령정보(N_IS_ISSUE)

In [ ]:
# # 1.8s
# isis = we.export_to_pandas("SELECT GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID FROM ODS_N_IS_ISSUE;")
# isis.info()

In [130]:
# about 2.1s
name = 'ODS_N_IS_ISSUE'
file_name = f'{name}.csv'
isis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['GNFD_NO', 'TY_STDR_ID', 'DNSTY_STDR_ID']) # usecols=['GNFD_NO', 'TY_STDR_ID', 'DNSTY_STDR_ID']
isis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GNFD_NO        1785 non-null   int64 
 1   DNSTY_STDR_ID  1785 non-null   object
 2   TY_STDR_ID     1785 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.0+ KB


In [131]:
isis.columns

Index(['GNFD_NO', 'DNSTY_STDR_ID', 'TY_STDR_ID'], dtype='object')

In [132]:
isis_ch_col = {
    'GNFD_NO':'발령번호', 
    'DNSTY_STDR_ID':'농도기준아이디', 
    'TY_STDR_ID':'유형기준아이디', 
}

In [133]:
isisr = isis.rename(columns=isis_ch_col)
isisr.columns

Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object')

In [134]:
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)

## N_IS_PENALTY

In [150]:
# 1.8s
ispe = ws.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY WHERE ROWNUM <= 10000;")
# ispe = ws.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY;")
ispe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   REGLT_NO  10000 non-null  int64
 1   REGLT_DE  10000 non-null  int64
dtypes: int64(2)
memory usage: 156.4 KB


In [151]:
# # about 2.1s
# name = 'N_IS_PENALTY'
# file_name = f'{name}.csv'
# ispe = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['REGLT_NO, 'REGLT_DE'], nrows=100**2, sep='|') # usecols=['REGLT_NO, 'REGLT_DE']
# ispe.info()

In [152]:
ispe.head()

,REGLT_NO,REGLT_DE
0,40995,20191210
1,40828,20191210
2,40676,20191210
3,40683,20191210
4,42850,20191210


In [153]:
ispe.columns

Index(['REGLT_NO', 'REGLT_DE'], dtype='object')

In [154]:
ispe_ch_col = {
    'REGLT_NO':'적발번호', 
    'REGLT_DE':'단속일', 
}

In [155]:
isper = ispe.rename(columns=ispe_ch_col)
isper.columns

Index(['적발번호', '단속일'], dtype='object')

In [156]:
isper.head()

,적발번호,단속일
0,40995,20191210
1,40828,20191210
2,40676,20191210
3,40683,20191210
4,42850,20191210


## 운행제한 단속정보(N_US_DISCLOSURE)

In [ ]:
# # 1.8s
# ispe = we.export_to_pandas("SELECT GNFD_NO, REGLT_DE FROM N_US_DISCLOSURE WHERE ROWNUM <= 10000;")
# # ispe = we.export_to_pandas("SELECT GNFD_NO, REGLT_DE FROM N_US_DISCLOSURE;")
# ispe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   GNFD_NO   10000 non-null  int64
 1   REGLT_DE  10000 non-null  int64
dtypes: int64(2)
memory usage: 156.4 KB


In [72]:
# about 2.1s
name = 'ODS_N_US_DISCLOSURE'
file_name = f'{name}.csv'
usdis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['NO', 'DISCL_TY', 'REGLT_AREA_CD', 'REG_SIDO_CD', 'REG_SIGNGU_CD'], nrows=100**2, ) # usecols=['NO', 'DISCL_TY', 'REGLT_AREA_CD', 'REG_SIDO_CD', 'REG_SIGNGU_CD']
usdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NO             10000 non-null  int64 
 1   REG_SIDO_CD    10000 non-null  int64 
 2   REG_SIGNGU_CD  10000 non-null  int64 
 3   DISCL_TY       10000 non-null  object
 4   REGLT_AREA_CD  10000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 390.8+ KB


In [82]:
usdis.head()

,NO,REG_SIDO_CD,REG_SIGNGU_CD,DISCL_TY,REGLT_AREA_CD
0,117112,41,110,S,41430
1,117111,11,0,S,41430
2,117107,41,410,S,41430
3,132833,11,0,S,41130
4,6933349,11,0,S,41130


In [73]:
usdis.columns

Index(['NO', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'DISCL_TY', 'REGLT_AREA_CD'], dtype='object')

In [76]:
usdis_ch_dict = {
    'NO':'번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'DISCL_TY':'적발유형', 
    'REGLT_AREA_CD':'단속지역코드', 
}

In [77]:
usdisr = usdis.rename(columns=usdis_ch_dict)
usdisr.columns

Index(['번호', '등록시도코드', '등록시군구코드', '적발유형', '단속지역코드'], dtype='object')

In [ ]:
# DISCL_TY(적발유형) : S(일반), H(고농도->상시)

## N_US_PENALTY

In [93]:
# 1.8s
uspe = ws.export_to_pandas('SELECT "NO", REGLT_CNT FROM N_US_PENALTY;')
uspe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443309 entries, 0 to 443308
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   NO         443309 non-null  int64
 1   REGLT_CNT  443309 non-null  int64
dtypes: int64(2)
memory usage: 6.8 MB


In [79]:
# about 2.1s
name = 'N_US_PENALTY'
file_name = f'{name}.csv'
uspe = pd.read_csv(os.path.join(ods_fold, file_name), nrows=100**2, sep='|' ) # usecols=['NO', 'REGLT_CNT']
uspe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ODR_YM           10000 non-null  int64  
 1   REGLT_YM         10000 non-null  int64  
 2   VHMNO            10000 non-null  object 
 3   VIN              10000 non-null  object 
 4   VHRNO            10000 non-null  object 
 5   REGLT_CNT        10000 non-null  int64  
 6   LSMTH_REGLT_CNT  10000 non-null  int64  
 7   LSMTH_BF_MM_CNT  10000 non-null  int64  
 8   LST_RQST_DT      0 non-null      float64
 9   REG_ID           10000 non-null  object 
 10  REG_DT           10000 non-null  object 
 11  SND_YN           10000 non-null  object 
 12  NO               10000 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 1015.8+ KB


In [94]:
uspe.head()

,NO,REGLT_CNT
0,8114417,1
1,6933701,4
2,6938411,10
3,8114173,1
4,6938670,14


In [95]:
uspe.columns

Index(['NO', 'REGLT_CNT'], dtype='object')

In [96]:
uspe_ch_dict = {
    'NO':'번호', 
    'REGLT_CNT':'적발건수', 
}

In [97]:
usper = uspe.rename(columns=uspe_ch_dict)
usper.columns

Index(['번호', '적발건수'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [98]:
carr.shape, len(carr['차대번호'].unique())

((2378646, 10), 2378598)

In [99]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(2378598, 10)

## 차종 코드 변환

In [100]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

## 용도 코드 변환

In [101]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [102]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1162311, 10)

## 연료 코드 변환

In [103]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '태양열', '경유 하이브리드', '등유', 'CNG 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [104]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [105]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1162311, 12)

## 저감장치구분 코드 변환

In [106]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1006', 'A1002', 'A1004', 'A1007'],
      dtype=object)

In [107]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '엔진교체', '2종', '1종+SCR', '삼원촉매'], dtype=object)

## 저감장치 부착 유무

In [108]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [109]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    531833
유      479113
Name: count, dtype: int64

## 등록&제원&저감이력 병합

In [110]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1162570, 13)

## 지역정보 병합

In [111]:
csac = csa.merge(coder, on='법정동코드', how='left')

### 매칭 안되는 지역 처리

In [112]:
csac.loc[csac['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동

## 조기폐차 정보들 병합

In [113]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1177453, 3)

In [114]:
elp.shape, len(elp['차대번호'].unique())

((1177453, 3), 1105195)

In [115]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1105195, 3)

In [116]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(844474, 3)

## 조기폐차 정보 추가

In [117]:
csac.shape

(1162570, 17)

In [118]:
df = csac.merge(elpm, on='차대번호', how='left')
df.shape

(1162570, 19)

## 비상시 및 계절제 병합

In [157]:
isdisr.shape, isisr.shape, isper.shape

((10000, 5), (1785, 3), (10000, 2))

In [158]:
isdisr.columns, isisr.columns, isper.columns

(Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '차대번호'], dtype='object'),
 Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object'),
 Index(['적발번호', '단속일'], dtype='object'))

In [159]:
isdisr.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,차대번호
0,54793831,1838,41,190,KMFXKS7BP3U657943
1,54793839,1838,41,190,KNAMA75432S388470
2,54793843,1838,28,237,KNGDNM9N14K178854
3,54795717,1838,41,130,KMFGA17PP7C072098
4,54793837,1838,41,190,KPBSH3AB18P082348


In [160]:
isdisr.shape

(10000, 5)

In [161]:
isdp= isdisr.merge(isper, on='적발번호', how='left')
isdp.shape

(10000, 6)

In [162]:
isdp.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,차대번호,단속일
0,54793831,1838,41,190,KMFXKS7BP3U657943,NaN
1,54793839,1838,41,190,KNAMA75432S388470,NaN
2,54793843,1838,28,237,KNGDNM9N14K178854,NaN
3,54795717,1838,41,130,KMFGA17PP7C072098,NaN
4,54793837,1838,41,190,KPBSH3AB18P082348,NaN


In [167]:
isdpi = isdp.merge(isisr, on='발령번호', how='left')
isdpi.shape

(10000, 8)

In [168]:
isdpi.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,차대번호,단속일,농도기준아이디,유형기준아이디
0,54793831,1838,41,190,KMFXKS7BP3U657943,NaN,C012,T001
1,54793839,1838,41,190,KNAMA75432S388470,NaN,C012,T001
2,54793843,1838,28,237,KNGDNM9N14K178854,NaN,C012,T001
3,54795717,1838,41,130,KMFGA17PP7C072098,NaN,C012,T001
4,54793837,1838,41,190,KPBSH3AB18P082348,NaN,C012,T001


### 지역정보 추가

In [169]:
coder.head()

,법정동코드,시도코드,시군구코드,시도,시군구
0,4171004506,41,710,경기도,양주군
1,4171004507,41,710,경기도,양주군
2,4171004508,41,710,경기도,양주군
3,4171004509,41,710,경기도,양주군
4,4171005000,41,710,경기도,양주군


In [173]:
coder_dup = coder.sort_values('법정동코드', ascending=True).drop_duplicates(['시도코드', '시군구코드']).reset_index(drop=True)
coder_dup.head()

,법정동코드,시도코드,시군구코드,시도,시군구
0,1100000000,11,0,서울특별시,NaN
1,1111000000,11,110,서울특별시,종로구
2,1114000000,11,140,서울특별시,중구
3,1117000000,11,170,서울특별시,용산구
4,1120000000,11,200,서울특별시,성동구


In [174]:
is_total = isdpi.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
is_total.shape

(10000, 12)

In [176]:
is_total.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,차대번호,단속일,농도기준아이디,유형기준아이디,시도코드,시군구코드,시도,시군구
0,54793831,1838,41,190,KMFXKS7BP3U657943,NaN,C012,T001,41,190,경기도,부천시
1,54793839,1838,41,190,KNAMA75432S388470,NaN,C012,T001,41,190,경기도,부천시
2,54793843,1838,28,237,KNGDNM9N14K178854,NaN,C012,T001,28,237,인천광역시,부평구
3,54795717,1838,41,130,KMFGA17PP7C072098,NaN,C012,T001,41,130,경기도,성남시
4,54793837,1838,41,190,KPBSH3AB18P082348,NaN,C012,T001,41,190,경기도,부천시


# Load

## 5등급 지역별 조기폐차

In [6]:
# about 2.1s
name = '5등급_조기폐차'
file_name = f'{name}.xlsx'
df1 = pd.read_excel(os.path.join(raw3_fold, file_name))
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 3.3+ KB


## 5등급 저공해 미조치

In [7]:
# about 2.1s
name = '5등급_저공해미조치'
file_name = f'{name}.xlsx'
no_dpf = pd.read_excel(os.path.join(raw3_fold, file_name))
no_dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      12 non-null     object
 1   구분      12 non-null     object
 2   차량대수    12 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 416.0+ bytes


## 등록&제원정보

In [9]:
# about 8m 53.6s
name = '[ODS]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
car = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29534047 entries, 0 to 29534046
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

## 등록&제원정보(5등급)

In [10]:
# about 37.6s
name = '[ODS][G5]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
g5 = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
g5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378648 entries, 0 to 2378647
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          float64
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          obje

## 저감장치 부착이력

In [66]:
# about 3s
name = '[ODS]저감장치부착이력(중복차대제거)(2022.12.28)'
file_name = f'{name}.csv'
att = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949', usecols=['차대번호', '저감장치구분'])
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    809255 non-null  object
 1   저감장치구분  809255 non-null  object
dtypes: object(2)
memory usage: 12.3+ MB


## 노후차 조기폐차 관리정보 수도권

In [11]:
# about 42.0s
name = '[ODS]노후차_조기폐차_관리정보_수도권(2022.12.28)'
file_name = f'{name}.csv'
aea = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770311 entries, 0 to 770310
Columns: 177 entries, ELPDSRC_MNGT_NO to PFMINPC_ELPDSRC_GNRLZ_RESN
dtypes: float64(56), int64(3), object(118)
memory usage: 1.0+ GB


In [12]:
aea.columns.to_list()

['ELPDSRC_MNGT_NO',
 '조기폐차상태코드',
 '저감장치구분',
 'INST_ID_CD',
 'OWNR_NM',
 'USER_BIZRNO',
 'USER_TELNO',
 'FXNUM',
 'USER_ZIP',
 'USER_1_ADRS',
 'USER_2_ADRS',
 '접수시도',
 'RCPT_SIGNGU',
 '차량등록번호',
 '차대번호',
 'VHCL_YRIDNW',
 'EGINCTY',
 'VHCL_PSCP',
 'VHCL_WGHT',
 'TOT_WGHT',
 'CRYNG_WGHT',
 'VHCNM',
 'VHCL_FRM',
 'VHCTY',
 'VHCL_PURPS',
 'FST_REG_DE',
 '폐차등록주행거리',
 'SCRCAR_ERA_DRVNG_DSTNC',
 'VHCL_FUEL',
 'PRCINSP_DE',
 'INSPOF_NM',
 'PRCINSP_JDGMNT_YN',
 'PRCINSP_RSLT_VAL',
 'PRCINSP_JDGMNT_STDR_VAL',
 'RCPT_DE',
 'INPT_DE',
 'RETR_NM',
 'SBSIDY_TRGT_YN',
 'ELPDSRC_TRGT_1_AT',
 'ELPDSRC_TRGT_2_AT',
 'ELPDSRC_TRGT_3_AT',
 'ELPDSRC_TRGT_4_AT',
 'ELPDSRC_TRGT_5_AT',
 'ELPDSRC_TRGT_6_AT',
 'ELPDSRC_TRGT_7_AT',
 'ELPDSRC_TRGT_CNFRM_REG_DE',
 'ELPDSRC_TRGT_CNFRM_REGR_NM',
 'PYMT_CNFRM_YN',
 'EXPC_SPRT_AMT',
 'DCIS_SPRT_AMT',
 'SBSIDY_UNPAY_RESN',
 'SBSIDY_RQEST_DE',
 'SBSIDY_RQEST_REGR_NM',
 'DOC_CNFRM_NO',
 'DOC_OUTT_DE',
 'INSP_RQT_SIWA_MTHD',
 'INSP_RQT_DE',
 'INSP_RQT_TM',
 'VRSC_ENTRPS_NM',

## 노후차 조기폐차 관리정보 수도권 외

In [13]:
# about 11.6s
name = '[ODS]노후차_조기폐차_관리정보_수도권외(2022.12.28)'
file_name = f'{name}.csv'
lgv = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291353 entries, 0 to 291352
Columns: 170 entries, ELPDSRC_MNGT_NO to PFMINPC_ELPDSRC_GNRLZ_RESN
dtypes: float64(71), int64(4), object(95)
memory usage: 377.9+ MB


In [14]:
lgv.columns.to_list()

['ELPDSRC_MNGT_NO',
 '조기폐차상태코드',
 '저감장치구분',
 'INST_ID_CD',
 'OWNR_NM',
 'USER_BIZRNO',
 'USER_TELNO',
 'FXNUM',
 'USER_ZIP',
 'USER_1_ADRS',
 'USER_2_ADRS',
 '접수시도',
 'RCPT_SIGNGU',
 '차량등록번호',
 '차대번호',
 'VHCL_YRIDNW',
 'EGINCTY',
 'VHCL_PSCP',
 'VHCL_WGHT',
 'TOT_WGHT',
 'CRYNG_WGHT',
 'VHCNM',
 'VHCL_FRM',
 'VHCTY',
 'VHCL_PURPS',
 'FST_REG_DE',
 '폐차등록주행거리',
 'SCRCAR_ERA_DRVNG_DSTNC',
 'VHCL_FUEL',
 'PRCINSP_DE',
 'INSPOF_NM',
 'PRCINSP_JDGMNT_YN',
 'PRCINSP_RSLT_VAL',
 'PRCINSP_JDGMNT_STDR_VAL',
 'RCPT_DE',
 'INPT_DE',
 'RETR_NM',
 'SBSIDY_TRGT_YN',
 'ELPDSRC_TRGT_1_AT',
 'ELPDSRC_TRGT_2_AT',
 'ELPDSRC_TRGT_3_AT',
 'ELPDSRC_TRGT_4_AT',
 'ELPDSRC_TRGT_5_AT',
 'ELPDSRC_TRGT_6_AT',
 'ELPDSRC_TRGT_7_AT',
 'ELPDSRC_TRGT_CNFRM_REG_DE',
 'ELPDSRC_TRGT_CNFRM_REGR_NM',
 'PYMT_CNFRM_YN',
 'EXPC_SPRT_AMT',
 'DCIS_SPRT_AMT',
 'SBSIDY_UNPAY_RESN',
 'SBSIDY_RQEST_DE',
 'SBSIDY_RQEST_REGR_NM',
 'DOC_CNFRM_NO',
 'DOC_OUTT_DE',
 'INSP_RQT_SIWA_MTHD',
 'INSP_RQT_DE',
 'INSP_RQT_TM',
 'VRSC_ENTRPS_NM',

## 법정동 코드

In [15]:
# about 5.6s
name = '법정동코드_2022.10.14(수정2023.05.04)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(big2_fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46342 entries, 0 to 46341
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46342 non-null  int64  
 1   시도        46342 non-null  object 
 2   시구군       46319 non-null  object 
 3   읍면동       45849 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46342 non-null  object 
 7   대기관리권역    46342 non-null  object 
 8   대기관리권역YN  46337 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [16]:
code.head()

,법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN
0,1100000000,서울특별시,NaN,NaN,NaN,NaN,존재,수도권,Y
1,1111000000,서울특별시,종로구,NaN,NaN,NaN,존재,수도권,Y
2,1111010100,서울특별시,종로구,청운동,NaN,NaN,존재,수도권,Y
3,1111010200,서울특별시,종로구,신교동,NaN,NaN,존재,수도권,Y
4,1111010300,서울특별시,종로구,궁정동,NaN,NaN,존재,수도권,Y


In [17]:
code = code.rename(columns={'법정동코드':'본거지법정동코드'})
code.head()

,본거지법정동코드,시도,시구군,읍면동,리,지역3,폐지여부,대기관리권역,대기관리권역YN
0,1100000000,서울특별시,NaN,NaN,NaN,NaN,존재,수도권,Y
1,1111000000,서울특별시,종로구,NaN,NaN,NaN,존재,수도권,Y
2,1111010100,서울특별시,종로구,청운동,NaN,NaN,존재,수도권,Y
3,1111010200,서울특별시,종로구,신교동,NaN,NaN,존재,수도권,Y
4,1111010300,서울특별시,종로구,궁정동,NaN,NaN,존재,수도권,Y


## 차명_연료_매칭

In [18]:
# about 0.1s
name = '차명_연료_매칭_2023.03.03_AM.11.12.03'
file_name = f'{name}.xlsx'
fuel = pd.read_excel(os.path.join(big2_fold, file_name))
fuel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   차명      216 non-null    object
 1   연료      216 non-null    object
dtypes: object(2)
memory usage: 3.5+ KB


In [19]:
fuel.head()

,차명,연료
0,BENZ S500,휘발유
1,BMW,휘발유
2,BMW 3-SERIES,휘발유
3,BMW 318I,휘발유
4,BMW 325CI,휘발유


## [로드] 운행제한 테이블(limit)

In [23]:
# about 2.1s
name = '차대번호별 적발'
file_name = f'{name}.csv'
limit = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False, encoding='cp949')
limit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731507 entries, 0 to 731506
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   VIN     731507 non-null  object 
 1   계절제_1차  37861 non-null   float64
 2   계절제_2차  54788 non-null   float64
 3   계절제_3차  68278 non-null   float64
 4   비상시     94084 non-null   float64
 5   상시      84185 non-null   float64
 6   계절제_4차  29307 non-null   float64
dtypes: float64(6), object(1)
memory usage: 39.1+ MB


In [24]:
limit.head()

,VIN,계절제_1차,계절제_2차,계절제_3차,비상시,상시,계절제_4차
0,KPBFA2AD14P100772,1.00,NaN,NaN,1.00,NaN,NaN
1,KMJWWH7JP5U656738,4.00,2.00,25.00,3.00,NaN,1.00
2,KNAJC52136A532787,2.00,NaN,NaN,1.00,NaN,NaN
3,KMHSH81WP6U081899,4.00,NaN,NaN,1.00,NaN,NaN
4,KNAJE55186K143510,1.00,NaN,NaN,1.00,NaN,NaN


## [로드] 계절제 적발지역별 차대번호(slimit)

In [25]:
# about 1.8s
name = '계절제 적발지역 차대번호'
file_name = f'{name}.csv'
slimit = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False, encoding='cp949')
slimit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163294 entries, 0 to 163293
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   적발지역    163294 non-null  object 
 1   적발시도    163294 non-null  object 
 2   차대번호    163294 non-null  object 
 3   계절제_1차  39111 non-null   float64
 4   계절제_2차  58088 non-null   float64
 5   계절제_3차  73775 non-null   float64
 6   계절제_4차  31673 non-null   float64
 7   차종      162378 non-null  object 
 8   차종유형    162378 non-null  object 
dtypes: float64(4), object(5)
memory usage: 11.2+ MB


# 전처리

## 말소 차량 제거

In [26]:
car['차량말소YN'].value_counts(dropna=False)

차량말소YN
N    25693532
Y     3840515
Name: count, dtype: int64

In [27]:
carm = car.loc[car['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(25693532, 49)

In [28]:
carm.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

In [29]:
g5['차량말소YN'].value_counts(dropna=False)

차량말소YN
Y    1223886
N    1154762
Name: count, dtype: int64

In [30]:
gm5 = g5.loc[g5['차량말소YN'] == 'N'].reset_index(drop=True)
gm5.shape

(1154762, 49)

In [31]:
gm5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

## 시도, 시구군 정보 부착

In [92]:
# about 21.0s
df = carm[['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명', '연료', '차량말소YN', '차종_등록정보', '차종유형(소분류)']].merge(code, on='본거지법정동코드', how='left')
df.shape

(25693532, 19)

In [93]:
df = df.rename(columns={'차종_등록정보':'차종', '차종유형(소분류)':'차종유형'})

In [94]:
df[['시도', '시구군']].isnull().sum()

시도      0
시구군    26
dtype: int64

In [95]:
df.loc[df['시도'].isnull() == True, ['본거지법정동코드', '시도', '시구군']]

,본거지법정동코드,시도,시구군


In [96]:
df.loc[df['시구군'].isnull() == True, ['본거지법정동코드', '시도', '시구군']]

,본거지법정동코드,시도,시구군
6197,4100000000,경기도,NaN
7508,3611000000,세종특별자치시,NaN
7600,3611000000,세종특별자치시,NaN
9991,3611000000,세종특별자치시,NaN
21115,3611000000,세종특별자치시,NaN
25219,3611000000,세종특별자치시,NaN
29709,3611000000,세종특별자치시,NaN
29767,3611000000,세종특별자치시,NaN
30062,3611000000,세종특별자치시,NaN
31443,3611000000,세종특별자치시,NaN


In [97]:
# about 0.8s
dfg5 = gm5[['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명', '연료', '차량말소YN']].merge(code, on='본거지법정동코드', how='left')
dfg5.shape

(1154762, 17)

In [98]:
dfg5[['시도', '시구군', '대기관리권역', '대기관리권역YN']].isnull().sum()

시도           0
시구군         21
대기관리권역       0
대기관리권역YN     0
dtype: int64

In [99]:
dfg5.loc[dfg5['시구군'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '대기관리권역', '대기관리권역YN']]

,본거지법정동코드,시도,시구군,대기관리권역,대기관리권역YN
5650,4100000000,경기도,NaN,수도권,Y
6873,3611000000,세종특별자치시,NaN,중부권,Y
6965,3611000000,세종특별자치시,NaN,중부권,Y
9268,3611000000,세종특별자치시,NaN,중부권,Y
19969,3611000000,세종특별자치시,NaN,중부권,Y
23989,3611000000,세종특별자치시,NaN,중부권,Y
28269,3611000000,세종특별자치시,NaN,중부권,Y
28327,3611000000,세종특별자치시,NaN,중부권,Y
28617,3611000000,세종특별자치시,NaN,중부권,Y
29961,3611000000,세종특별자치시,NaN,중부권,Y


## 연료 결측, 기타연료 확인

In [100]:
df['연료'].value_counts(dropna=False)

연료
휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
전기               368693
NaN              146798
경유 하이브리드          35779
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [101]:
# fuel_dict = {a:b for a, b in fuel.values}
# fuel_dict

In [102]:
df.loc[((df['연료'].isnull() == True) | (df['연료'] == '기타연료')), '차명'].unique()

array(['트레일러', '미스비시샤시(세미트레일러)', '도큐트레일러', ..., '대흥18.8톤가변형저상트레일러',
       '캠핑플레인23.2톤가변식저상트레일러', '에이사캠핑트레일러'], dtype=object)

In [103]:
# for one in df.loc[((df['연료'].isnull() == True) | (df['연료'] == '기타연료')), '차명'].unique():
#     df.loc[((df['연료'].isnull() == True) | (df['연료'] == '기타연료')) & (df['차명'] == one), '연료'] = fuel_dict[one]

In [104]:
df['연료'].value_counts(dropna=False)

연료
휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
전기               368693
NaN              146798
경유 하이브리드          35779
CNG(압축천연가스)       30461
수소                28227
LPG 하이브리드         14352
기타연료                933
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: count, dtype: int64

In [105]:
# df['연료'].value_counts(dropna=False).reset_index().to_excel(os.path.join(analysis_fold, '연료별_수.xlsx'), index=False)

In [106]:
df['배출가스등급'].value_counts(dropna=False)

배출가스등급
2     10960244
3      8623130
1      2771395
4      1511214
5      1154733
X       288966
T2      247483
T3      136301
T4          37
T5          29
Name: count, dtype: int64

In [107]:
df['배출가스등급'].value_counts(dropna=False).reset_index()

,배출가스등급,count
0,2,10960244
1,3,8623130
2,1,2771395
3,4,1511214
4,5,1154733
5,X,288966
6,T2,247483
7,T3,136301
8,T4,37
9,T5,29


In [108]:
# df['배출가스등급'].value_counts(dropna=False).reset_index().to_excel(os.path.join(analysis_fold, '배출가스등급_개수.xlsx'), index=False)

In [109]:
dfg5['연료'].value_counts(dropna=False)

연료
경유             1124215
휘발유              30051
LPG(액화석유가스)        436
NaN                 29
기타연료                25
CNG(압축천연가스)          5
알코올                  1
Name: count, dtype: int64

In [110]:
fuel_dict = {a:b for a, b in fuel.values}
fuel_dict

{'BENZ S500': '휘발유',
 'BMW': '휘발유',
 'BMW 3-SERIES': '휘발유',
 'BMW 318I': '휘발유',
 'BMW 325CI': '휘발유',
 'BMW 325I': '휘발유',
 'BMW 325i': '휘발유',
 'BMW 328I': '휘발유',
 'BMW 525I': '휘발유',
 'BMW 528I': '휘발유',
 'BMW 530I': '휘발유',
 'BMW 5SERIES': '휘발유',
 'BMW 750Li': '휘발유',
 'BMW X3': '휘발유',
 'BMW X5': '휘발유',
 'BMW Z3': '휘발유',
 'BMW530I': '휘발유',
 'BUICK CENTURY': '휘발유',
 'BUICK LESABRE': '휘발유',
 'BUICK TERRAZA': '휘발유',
 'CADILLAC CTS': '휘발유',
 'CHEVROLET IMPALA': '휘발유',
 'CHEVROLET PRIZM': '휘발유',
 'CHRYSLER': '휘발유',
 'CHRYSLER 300 LTD': '휘발유',
 'CHRYSLER 300C': '경유',
 'CHRYSLER HEMI AWD': '휘발유',
 'CHRYSLER PT CRUISER': '휘발유',
 'CHRYSLER TOWN & COUNTRY(7)': '휘발유',
 'CIELO': '휘발유',
 'CORSICA': '휘발유',
 'COUGAR': '휘발유',
 'DODGE CALIBER': '휘발유',
 'DODGE CARAVAN': '휘발유',
 'DODGE CHARGER': '휘발유',
 'DODGE DAKOTA': '휘발유',
 'DODGE DURANGO': '휘발유',
 'DODGE GRAND CARAVAN': '휘발유',
 'DODGE NEON SXT': '휘발유',
 'EF소나타': '휘발유',
 'ESCORT': '휘발유',
 'FIREBIRD': '휘발유',
 'FORD ESCAPE': '휘발유',
 'FORD EXPEDITION': '휘발유'

In [111]:
# about 7.8s
for one in dfg5.loc[((dfg5['연료'].isnull() == True) | (dfg5['연료'] == '기타연료')), '차명'].unique():
    dfg5.loc[((dfg5['연료'].isnull() == True) | (dfg5['연료'] == '기타연료')) & (dfg5['차명'] == one), '연료'] = fuel_dict[one]

In [112]:
dfg5['연료'].value_counts(dropna=False)

연료
경유             1124238
휘발유              30075
LPG(액화석유가스)        438
무동력                  5
CNG(압축천연가스)          5
알코올                  1
Name: count, dtype: int64

## 조기폐차 정보 병합

In [113]:
aear = aea[['차대번호', '조기폐차최종승인YN']]
lgvr = lgv[['차대번호', '조기폐차최종승인YN']]

In [114]:
early = pd.concat([aear, lgvr], ignore_index=True)
early.shape

(1061664, 2)

In [115]:
elp = early.drop_duplicates('차대번호').reset_index(drop=True)
elp.shape

(999576, 2)

In [116]:
df.shape

(25693532, 19)

In [117]:
dfm = df.merge(elp, on='차대번호', how='left')
dfm.shape

(25693532, 20)

In [118]:
dfg5.shape

(1154762, 17)

In [119]:
dfg5m = dfg5.merge(elp, on='차대번호', how='left')
dfg5m.shape

(1154762, 18)

In [120]:
dfm.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '차종', '차종유형', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '조기폐차최종승인YN'],
      dtype='object')

In [121]:
dfg5m.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역',
       '대기관리권역YN', '조기폐차최종승인YN'],
      dtype='object')

## 저감장치구분 변환

In [122]:
att['저감장치구분'].unique()

array(['엔진교체', '3종', '1종', '2종', '엔진개조', '1종+SCR', '삼원촉매'], dtype=object)

In [123]:
att.loc[att['저감장치구분'] == '1종', '저감장치'] = 'DPF'
att.loc[att['저감장치구분'] == '1종+SCR', '저감장치'] = 'DPF'

In [124]:
df = dfm.merge(att, on='차대번호', how='left')
dfg5 = dfg5m.merge(att, on='차대번호', how='left')

In [125]:
df.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '차종', '차종유형', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '조기폐차최종승인YN', '저감장치구분', '저감장치'],
      dtype='object')

In [126]:
dfg5.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역',
       '대기관리권역YN', '조기폐차최종승인YN', '저감장치구분', '저감장치'],
      dtype='object')

# 분석

## 전체

### 내연기관차

In [106]:
# about 11.8s
gas_car = df.loc[(df['연료'].isnull() == False) & (df['연료'] != '전기') & (df['연료'] != '수소') & (df['연료'] != '기타연료') & (df['연료'] != '태양열')]
gas_car['연료'].value_counts(dropna=False)

휘발유            12177033
경유              9890990
LPG(액화석유가스)     1904188
휘발유 하이브리드       1095741
경유 하이브리드          35779
CNG(압축천연가스)       30461
LPG 하이브리드         14352
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
등유                    1
Name: 연료, dtype: int64

In [107]:
# 내연기관차
stat_gas1 = gas_car.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
stat_gas1

대기관리권역YN          N          Y  NaN
대기관리권역 시도                          
그 외    강원 835141.00        NaN  NaN
       제주 669165.00        NaN  NaN
남부권    광주       NaN  704482.00  NaN
       전남 614500.00  584570.00  NaN
동남권    경남 533776.00 1332311.00  NaN
       경북 576666.00  927885.00  NaN
       대구       NaN 1210706.00  NaN
       부산       NaN 1476574.00  NaN
       울산       NaN  581521.00  NaN
수도권    경기 127253.00 6172880.00 1.00
       서울       NaN 3166381.00  NaN
       인천   9045.00 1695216.00  NaN
중부권    대전       NaN  693620.00  NaN
       세종       NaN  189726.00  NaN
       전북 340496.00  622384.00  NaN
       충남  31067.00 1156172.00  NaN
       충북 118967.00  778374.00  NaN

In [66]:
# 내연기관차
stat_gas2 = gas_car.groupby(['대기관리권역', '시도', '배출가스등급', '대기관리권역YN'])['차대번호'].count().unstack()
stat_gas2

대기관리권역YN                 N        Y
대기관리권역 시도 배출가스등급                   
그 외    강원 1       62814.00      NaN
          2      349994.00      NaN
          3      304509.00      NaN
          4       56327.00      NaN
          5       48547.00      NaN
...                    ...      ...
중부권    충북 5       10468.00 32099.00
          T2        389.00  4602.00
          T3        617.00  4062.00
          T5           NaN     2.00
          X         419.00  3114.00

[156 rows x 2 columns]

In [67]:
# 내연기관차
stat_gas3 = gas_car.groupby(['대기관리권역', '시도', '연료', '대기관리권역YN'])['차대번호'].count().unstack()
stat_gas3

대기관리권역YN                      N         Y
대기관리권역 시도 연료                             
그 외    강원 CNG(압축천연가스)    342.00       NaN
          LPG 하이브리드      594.00       NaN
          LPG(액화석유가스)  57868.00       NaN
          경유          372822.00       NaN
          경유 하이브리드       399.00       NaN
...                         ...       ...
중부권    충북 LPG(액화석유가스)   8641.00  61573.00
          경유           62530.00 319859.00
          경유 하이브리드        41.00    648.00
          휘발유          44932.00 368498.00
          휘발유 하이브리드     2726.00  26742.00

[143 rows x 2 columns]

In [68]:
# 내연기관차
stat_gas4 = gas_car.groupby(['대기관리권역', '시도', '배출가스등급', '연료', '대기관리권역YN'])['차대번호'].count().unstack()
stat_gas4

대기관리권역YN                            N       Y
대기관리권역 시도 배출가스등급 연료                          
그 외    강원 1      LPG 하이브리드     594.00     NaN
                 LPG(액화석유가스) 13209.00     NaN
                 휘발유         24236.00     NaN
                 휘발유 하이브리드   24775.00     NaN
          2      CNG(압축천연가스)   203.00     NaN
...                               ...     ...
중부권    충북 X      LPG(액화석유가스)    15.00  226.00
                 경유            172.00  767.00
                 경유 하이브리드        4.00   19.00
                 휘발유           163.00 1570.00
                 휘발유 하이브리드      65.00  530.00

[593 rows x 2 columns]

#### [출력] 내연기관차 대기관리권역별 등급별(말소 제거) 차량 현황

In [62]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.05.04_AM.10.46.25'

In [73]:
with pd.ExcelWriter(os.path.join(analysis_fold, f'통계_내연기관차_대기관리권역_등급_연료별({today_date}).xlsx')) as writer:
    stat_gas1.to_excel(writer, sheet_name='대기관리권역')
    stat_gas2.to_excel(writer, sheet_name='대기관리권역_등급')
    stat_gas3.to_excel(writer, sheet_name='대기관리권역_연료')
    stat_gas4.to_excel(writer, sheet_name='대기관리권역_등급_연료')

### 무공해차(전기, 수소, 태양열)

In [54]:
# about 12.1s
no_car = df.loc[(df['연료'].isnull() == False) & ((df['연료'] == '전기') | (df['연료'] == '수소') | (df['연료'] == '태양열'))]
no_car['연료'].value_counts(dropna=False)

전기     368693
수소      28227
태양열         2
Name: 연료, dtype: int64

In [64]:
# 내연기관차
stat_no1 = no_car.groupby(['대기관리권역', '시도', '대기관리권역YN'])['차대번호'].count().unstack()
stat_no1

대기관리권역YN         N        Y
대기관리권역 시도                  
그 외    강원 15551.00      NaN
       제주 31714.00      NaN
남부권    광주      NaN  9578.00
       전남  6381.00  8952.00
동남권    경남  5797.00 17641.00
       경북  7434.00 10742.00
       대구      NaN 23647.00
       부산      NaN 22518.00
       울산      NaN  7203.00
수도권    경기  1868.00 76140.00
       서울      NaN 61017.00
       인천    51.00 25443.00
중부권    대전      NaN 14670.00
       세종      NaN  3028.00
       전북  5207.00  8976.00
       충남   434.00 16803.00
       충북  1890.00 14237.00

In [70]:
# 내연기관차
stat_no2 = no_car.groupby(['대기관리권역', '시도', '배출가스등급', '대기관리권역YN'])['차대번호'].count().unstack()
stat_no2

대기관리권역YN                N        Y
대기관리권역 시도 배출가스등급                  
그 외    강원 1      15487.00      NaN
          X         64.00      NaN
       제주 1      31654.00      NaN
          X         60.00      NaN
남부권    광주 1           NaN  9552.00
          X           NaN    26.00
       전남 1       6349.00  8904.00
          X         32.00    48.00
동남권    경남 1       5758.00 17552.00
          X         39.00    89.00
       경북 1       7381.00 10716.00
          X         53.00    26.00
       대구 1           NaN 23556.00
          X           NaN    91.00
       부산 1           NaN 22396.00
          X           NaN   122.00
       울산 1           NaN  7202.00
          X           NaN     1.00
수도권    경기 1       1861.00 75600.00
          X          7.00   540.00
       서울 1           NaN 60655.00
          X           NaN   362.00
       인천 1         51.00 25264.00
          X           NaN   179.00
중부권    대전 1           NaN 14624.00
          X           NaN    46.00
       세종 1           NaN  3025.00
          X           NaN     3.00
       전북 1       5190.00  8912.00
          X         17.00    64.00
       충남 1        432.00 16747.00
          X          2.00    56.00
       충북 1       1879.00 14163.00
          X         11.00    74.00

In [71]:
# 내연기관차
stat_no3 = no_car.groupby(['대기관리권역', '시도', '연료', '대기관리권역YN'])['차대번호'].count().unstack()
stat_no3

대기관리권역YN             N        Y
대기관리권역 시도 연료                   
그 외    강원 수소   2504.00      NaN
          전기  13047.00      NaN
       제주 수소      5.00      NaN
          전기  31709.00      NaN
남부권    광주 수소       NaN  1096.00
          전기       NaN  8482.00
       전남 수소     84.00   627.00
          전기   6297.00  8325.00
동남권    경남 수소    207.00  2182.00
          전기   5590.00 15459.00
       경북 수소     68.00   148.00
          전기   7366.00 10594.00
       대구 수소       NaN   529.00
          전기       NaN 23118.00
       부산 수소       NaN  1887.00
          전기       NaN 20631.00
       울산 수소       NaN  2438.00
          전기       NaN  4765.00
수도권    경기 수소     26.00  5989.00
          전기   1842.00 70149.00
          태양열      NaN     2.00
       서울 수소       NaN  2839.00
          전기       NaN 58178.00
       인천 수소      1.00  1569.00
          전기     50.00 23874.00
중부권    대전 수소       NaN  1208.00
          전기       NaN 13462.00
       세종 수소       NaN   242.00
          전기       NaN  2786.00
       전북 수소    420.00  1270.00
          전기   4787.00  7706.00
       충남 수소      8.00  1182.00
          전기    426.00 15621.00
       충북 수소     64.00  1634.00
          전기   1826.00 12603.00

In [72]:
# 내연기관차
stat_no4 = no_car.groupby(['대기관리권역', '시도', '배출가스등급', '연료', '대기관리권역YN'])['차대번호'].count().unstack()
stat_no4

대기관리권역YN                   N        Y
대기관리권역 시도 배출가스등급 연료                  
그 외    강원 1      수소  2496.00      NaN
                 전기 12991.00      NaN
          X      수소     8.00      NaN
                 전기    56.00      NaN
       제주 1      수소     5.00      NaN
...                      ...      ...
중부권    충남 X      전기     2.00    50.00
       충북 1      수소    64.00  1618.00
                 전기  1815.00 12545.00
          X      수소      NaN    16.00
                 전기    11.00    58.00

[65 rows x 2 columns]

#### [출력] 무공해차 대기관리권역별 등급별(말소 제거) 차량 현황

In [74]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.05.04_AM.10.50.07'

In [75]:
with pd.ExcelWriter(os.path.join(analysis_fold, f'통계_무공해차_대기관리권역_등급_연료별({today_date}).xlsx')) as writer:
    stat_no1.to_excel(writer, sheet_name='대기관리권역')
    stat_no2.to_excel(writer, sheet_name='대기관리권역_등급')
    stat_no3.to_excel(writer, sheet_name='대기관리권역_연료')
    stat_no4.to_excel(writer, sheet_name='대기관리권역_등급_연료')

## 5등급

### 5등급 지역별 조기폐차 현황

In [ ]:
dfg5.columns

In [9]:
STD_BD_GRD5_ELPDSRC_CURSTT = df1.copy()

#### [출력] STD_BD_GRD5_ELPDSRC_CURSTT

In [ ]:
# expdf = STD_BD_GRD5_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 5등급 지역별 저공해미조치 차량현황

In [10]:
cdict = {
    '지역':'RGN',
    '구분':'SEASON',
    '차량대수':'VHCL_MKCNT', 
}

In [11]:
STD_BD_GRD5_LEM_N_MOD = no_dpf.rename(columns=cdict)
STD_BD_GRD5_LEM_N_MOD.columns

Index(['RGN', 'SEASON', 'VHCL_MKCNT'], dtype='object')

#### [출력] STD_BD_GRD5_LEM_N_MOD

In [ ]:
# expdf = STD_BD_GRD5_LEM_N_MOD
# table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 지역별 계절관리제 적발 현황

In [127]:
limit.head()

,차대번호,계절제_1차,계절제_2차,계절제_3차,비상시,상시,계절제_4차
0,KPBFA2AD14P100772,1.00,NaN,NaN,1.00,NaN,NaN
1,KMJWWH7JP5U656738,4.00,2.00,25.00,3.00,NaN,1.00
2,KNAJC52136A532787,2.00,NaN,NaN,1.00,NaN,NaN
3,KMHSH81WP6U081899,4.00,NaN,NaN,1.00,NaN,NaN
4,KNAJE55186K143510,1.00,NaN,NaN,1.00,NaN,NaN


In [128]:
limit = limit.rename(columns={'VIN':'차대번호'})

In [129]:
df.shape

(25693532, 22)

In [130]:
lmt1 = df.merge(limit, on='차대번호', how='left')
lmt1.shape

(25693532, 28)

In [131]:
lmt1.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '차종', '차종유형', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '조기폐차최종승인YN', '저감장치구분', '저감장치', '계절제_1차',
       '계절제_2차', '계절제_3차', '비상시', '상시', '계절제_4차'],
      dtype='object')

In [132]:
lmt1['시도'].unique()

array(['서울특별시', '대구광역시', '인천광역시', '강원도', '경상남도', '전라북도', '충청북도', '경기도',
       '부산광역시', '대전광역시', '울산광역시', '경상북도', '제주특별자치도', '전라남도', '광주광역시',
       '충청남도', '세종특별자치시'], dtype=object)

In [133]:
lmt1.loc[(lmt1['시도'] == '서울특별시') | (lmt1['시도'] == '경기도') | (lmt1['시도'] == '인천광역시'), '지역'] = '수도권'
lmt1['지역'] = lmt1['지역'].fillna('수도권외')

In [134]:
lmt1['지역'].unique()

array(['수도권', '수도권외'], dtype=object)

In [136]:
lmt1.loc[lmt1['저감장치'] == 'DPF', '저감장치부착유무'] = '유'
lmt1['저감장치부착유무'] = lmt1['저감장치부착유무'].fillna('무')

In [137]:
lmt1.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '차종', '차종유형', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '조기폐차최종승인YN', '저감장치구분', '저감장치', '계절제_1차',
       '계절제_2차', '계절제_3차', '비상시', '상시', '계절제_4차', '지역', '저감장치부착유무'],
      dtype='object')

In [138]:
season = lmt1.loc[ (lmt1['계절제_1차'] > 0) | (lmt1['계절제_2차'] > 0) | (lmt1['계절제_3차'] > 0) | (lmt1['계절제_4차'] > 0),[
    '차대번호', 
    '계절제_1차', 
    '계절제_2차',
    '계절제_3차', 
    '계절제_4차',
    '지역', 
    '시도', 
    '저감장치부착유무', 
    '차종', 
    '차종유형', 
]]
season.shape

(95688, 10)

In [139]:
cdict = {
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS',
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS',
    '지역':'RGN', 
    '시도':'CTPV', 
    '저감장치부착유무':'DPF_EXTRNS_YN', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
}

In [141]:
STD_BD_SEASON_CRDN_NOCS_CURSTT = season.rename(columns=cdict)
STD_BD_SEASON_CRDN_NOCS_CURSTT.columns

Index(['VIN', 'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS', 'RGN', 'CTPV',
       'DPF_EXTRNS_YN', 'VHCTY_CD', 'VHCTY_TY'],
      dtype='object')

#### [출력] STD_BD_SEASON_CRDN_NOCS_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
# table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 지역별 상시운행제한 단속 현황

In [ ]:
# orditm = lmt1.loc[lmt1['상시'] > 0, [
#     '차대번호',
#     '적발년도',
#     '적발건수',
#     '지역',
#     '시도',
# ]]

#### [로드]

In [145]:
# about 2.1s
name = '상시 적발지역 차대번호2'
file_name = f'{name}.csv'
orditm = pd.read_csv(os.path.join(raw3_fold, file_name), encoding='cp949')
orditm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106472 entries, 0 to 106471
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    106472 non-null  object
 1   적발년도    106472 non-null  int64 
 2   적발건수    106472 non-null  int64 
 3   지역      106472 non-null  object
 4   시도      106472 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.1+ MB


In [146]:
cdict = {
    '차대번호':'VIN', 
    '적발년도':'DSCL_YR', 
    '적발건수':'DSCL_NOCS', 
    '지역':'RGN',
    '시도':'CTPV', 
}

In [147]:
STD_BD_ORDITM_DSCL_CURSTT = orditm.rename(columns=cdict)
STD_BD_ORDITM_DSCL_CURSTT.columns

Index(['VIN', 'DSCL_YR', 'DSCL_NOCS', 'RGN', 'CTPV'], dtype='object')

#### [출력] STD_BD_ORDITM_DSCL_CURSTT

In [ ]:
# expdf = STD_BD_ORDITM_DSCL_CURSTT
# table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 적발지역별 계절제 단속 현황 출력

In [148]:
slimit.head()

,적발지역,적발시도,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,차종,차종유형
0,수도권,서울,KMFZCS7JP7U292779,4.00,NaN,NaN,NaN,화물,소형
1,수도권,인천,KMHSD81VP2U244691,1.00,3.00,NaN,NaN,승용,중형
2,수도권,인천,KMHJN81VP6U261521,1.00,NaN,NaN,NaN,승용,중형
3,수도권,인천,KNAJC52135A471475,1.00,NaN,NaN,NaN,승용,대형
4,수도권,경기,KMHSJ81WP6U029832,1.00,4.00,NaN,NaN,승용,대형


In [149]:
slimit.columns

Index(['적발지역', '적발시도', '차대번호', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '차종',
       '차종유형'],
      dtype='object')

In [150]:
cdict = {
    '적발지역':'DSCL_RGN', 
    '적발시도':'DSCL_CTPV', 
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS', 
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS', 
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
}

In [151]:
STD_BD_SEASON_DSCL_RGN_CURSTT = slimit.rename(columns=cdict)
STD_BD_SEASON_DSCL_RGN_CURSTT.columns

Index(['DSCL_RGN', 'DSCL_CTPV', 'VIN', 'SEASON_1ODR_CRDN_NOCS',
       'SEASON_2ODR_CRDN_NOCS', 'SEASON_3ODR_CRDN_NOCS',
       'SEASON_4ODR_CRDN_NOCS', 'VHCTY_CD', 'VHCTY_TY'],
      dtype='object')

#### [출력] STD_BD_SEASON_DSCL_RGN_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
# table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

# code end